<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

df = pd.read_csv('/datasets/insurance.csv')
df.info(), df.head(), df.shape, df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


(None,
    Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
 0    1     41.0   49600.0            1                  0
 1    0     46.0   38000.0            1                  1
 2    0     29.0   21000.0            0                  0
 3    0     21.0   41700.0            2                  0
 4    1     28.0   26100.0            0                  0,
 (5000, 5),
                Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
 count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
 mean      0.499000    30.952800  39916.360000     1.194200           0.148000
 std       0.500049     8.440807   9900.083569     1.091387           0.463183
 min       0.000000    18.000000   5300.000000     0.000000           0.000000
 25%       0.000000    24.000000  33300.000000     0.000000           0.000000
 50%       0.000000    30.000000  40200.000000     1.000000           0.000000
 75%       1.000000    37.000000  46600.000000     2.000000           0.0

В данных пропусков нет. Целевым признаком определим "Страховые выплаты". Для решения категориальной задачи выбираем модель Линейной регрессии.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Создадим модель Линейной регрессии с парамтрами w и w0, расчитаем предсказания и проверим метрикой r2 точность модели

In [2]:
#сгенерировать случайную матрицу функцией numpy.random.normal()
#найти обратную матрицу, вызвать функцию numpy.linalg.inv() 

features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


Чтобы проверить теорию умножения матриц, создадим случайную матрицу, конечно квадратную и провеми ее на обратимость.

In [3]:
e = np.random.random((4,4))
print(e)

[[0.06604878 0.5335151  0.78555733 0.67658776]
 [0.6538715  0.28270238 0.09467045 0.30315422]
 [0.37013132 0.40863205 0.59659196 0.16899082]
 [0.97903941 0.40128844 0.81556935 0.60628743]]


In [4]:
print(np.linalg.inv(e).dot(e))

[[ 1.00000000e+00  2.83355121e-17  1.65739283e-16  5.46539955e-17]
 [-4.14009009e-16  1.00000000e+00 -1.94453390e-17 -5.18251869e-16]
 [-1.04248300e-18  9.96301248e-17  1.00000000e+00  1.91540202e-16]
 [-3.31687622e-18  1.49881984e-17  7.40157423e-17  1.00000000e+00]]


уножение матрицы на обратную себе привело к единичной матрице что свидетельствует об обратимости нашей случайной матрицы

In [5]:
# умножим матрицы
features_e = np.dot(features, e)
print(features_e)
features_e.shape

[[18386.36737288 20280.67530472 29596.44398795  8395.65676554]
 [14096.04735066 15541.42351594 22675.66501017  6436.20246131]
 [ 7791.72002791  8589.47142897 12531.17666942  3557.59864803]
 ...
 [12562.48731229 13859.08313564 20227.9920989   5736.06438574]
 [12120.68255753 13370.22488335 19513.87221068  5535.16458789]
 [15046.68514863 16599.31171952 24225.88560365  6870.79839969]]


(5000, 4)

In [6]:
# проверим метрику r2 на перемноженной матрице
model.fit(features_e, target)
predictions = model.predict(features_e)
print(r2_score(target, predictions))

0.42494550286664123


качество модели не изменилось

**Ответ:** При умножении показателей на обратимую матрицу показатель Линейной регрессии не изменится.

**Обоснование:** В нашей формуле обучения используется обратная матрица:
$$
w = (X^T X)^{-1} X^T y
$$
Если наша исходная матрица умножена на другую обратимую матрицу $P$ мы получим:
$$
w = P^{-1}(X^T X)^{-1} (P^T)^{-1} P^TX^T y
$$
$$
w = P^{-1}(X^T X)^{-1} EX^T y
$$
предсказание вычисляем по формуле:
$$
a = XPw
$$
тогда подставив значение веса в формулу предсказания получим
$$
a = P^{-1}PX(X^T X)^{-1} EX^T y
$$
при перемножении матрицы $P$ на обратную $P^{-1}$ это приводит к единичной матрице, а если любую матрицу умножить на единичную, результатом будет исходная матрица.
$$
X^{-1} X = E
$$В итоге получаем $a = EX(X^T X)^{-1} EX^T y$ , E сокращаем

## Алгоритм преобразования

**Алгоритм**

Проверим преобразование на модели Линейной регрессии из библиотеки sklern:
* разобьем данные на тренировочную и тестовую
* выделим обучающую выборку и целевую
* обучим модель на исходных данных
* проверим метрикой R2 качество модели
* перемножим исходные данные на случайную обратимую матрицу
* проверим качество полученных предсказаний метрикой R2

**Обоснование**

Ожидаем получить метрику R2 на преобразованных данных близкую к полученным данным на исходных данных, так как по теории умножение на обратимую матрицу должно привести к схожему результату.

## Проверка алгоритма

In [7]:
train, valid = train_test_split(df, test_size=0.25, random_state=12345)
# разделим данные на тренировочную и проверочную
features_train = train.drop('Страховые выплаты', axis=1)
target_train = train['Страховые выплаты']
features_valid = valid.drop('Страховые выплаты', axis=1)
target_valid = valid['Страховые выплаты']
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print(r2_score(target_valid, predictions))

0.4352275712702667


In [12]:
# умножим тренировочные данные на обратимую матрицу
#df_e = np.dot(df, e)
#train_e, valid_e = train_test_split(df_e, test_size=0.25, random_state=12345)
# разделим данные на тренировочную и проверочную
#features_train_e = train_e.drop('Страховые выплаты', axis=1)
##target_train_e = train_e['Страховые выплаты']
#features_valid_e = valid_e.drop('Страховые выплаты', axis=1)
#target_valid_e = valid_e['Страховые выплаты']
features_train_e = np.dot(features_train, e)
features_valid_e = np.dot(features_valid, e)
print(features_train_e)
features_train_e.shape

[[13427.91540599 14805.57123305 21602.30102246  6131.78611422]
 [21341.86180602 23547.35150349 34367.70143076  9744.85494174]
 [15234.30024668 16804.22503917 24523.7747097   6955.82983774]
 ...
 [16572.72389687 18278.37825737 26673.14336845  7567.60175787]
 [18561.88054459 20480.29033503 29894.60238175  8475.53451889]
 [15113.78148123 16677.5590047  24342.75083557  6900.58530013]]


(3750, 4)

In [14]:
# проверим метрику r2 на перемноженной матрице
model.fit(features_train_e, target_train)
predictions = model.predict(features_valid_e)
print(r2_score(target_valid, predictions))

0.4352275742216486


Качество метрики мы получили точно такое же как и до шифрования данных.

**Вывод**
Мы проверили качество модели на исходных данных и на зашифрованных путем перемножения на обратимую матрицу. Результат оказался идентичным.


Используя модель Линейной регрессии мы можем данные пользователей хранить в виде данных перемноженных на обратимую матрицу, это не повлияет на качество предсказаний нашей модели.